In [32]:
import torch
import os
from torchvision import datasets, transforms, models

In [33]:
train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')

train_path = os.path.join(train_dir, '1_train_data.tar.pth')
eval_path = os.path.join(eval_dir, '1_eval_data.tar.pth')

t = torch.load(train_path, weights_only = False)


In [34]:
from torchvision import models
import torch

# Load a pre-trained ResNet model
resnet =  models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove the last layer
resnet.eval()  # Set to evaluation mode

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)


In [35]:
import numpy as np
import torch

# Example ndarray of shape (2500, 32, 32, 3)
data, targets = t['data'], t['targets'] # both numpy.ndarray
# Convert to PyTorch tensor
X_tensor = torch.tensor(data, dtype=torch.float32)  # Convert to tensor
X_tensor = X_tensor.permute(0, 3, 1, 2)  # Change shape to (2500, 3, 32, 32)


In [36]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (ResNet input size)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

tensor = X_tensor.float()

transformed_images = []
for image in tensor:
    # Convert each image tensor (C, H, W) to PIL Image for transformation
    transformed_image = transform(image)  # Apply the transformations
    transformed_images.append(transformed_image)

# 4. Stack the transformed images back into a batch
preprocessed_tensor = torch.stack(transformed_images)  # Shape: (2500, 3, 224, 224)

# 5. Check the shape of the preprocessed tensor
print(preprocessed_tensor.shape)  

torch.Size([2500, 3, 224, 224])


In [49]:
transform(tensor)

tensor([[[[ 255.5240,  255.5240,  255.5240,  ...,  644.1703,  644.1703,
            644.1703],
          [ 255.5240,  255.5240,  255.5240,  ...,  644.1703,  644.1703,
            644.1703],
          [ 255.5240,  255.5240,  255.5240,  ...,  644.1703,  644.1703,
            644.1703],
          ...,
          [ 770.8079,  770.8079,  770.8079,  ...,  535.0000,  535.0000,
            535.0000],
          [ 770.8079,  770.8079,  770.8079,  ...,  535.0000,  535.0000,
            535.0000],
          [ 770.8079,  770.8079,  770.8079,  ...,  535.0000,  535.0000,
            535.0000]],

         [[ 274.7500,  274.7500,  274.7500,  ...,  551.5357,  551.5357,
            551.5357],
          [ 274.7500,  274.7500,  274.7500,  ...,  551.5357,  551.5357,
            551.5357],
          [ 274.7500,  274.7500,  274.7500,  ...,  551.5357,  551.5357,
            551.5357],
          ...,
          [ 640.8214,  640.8214,  640.8214,  ...,  408.6786,  408.6786,
            408.6786],
          [ 640.82

In [39]:
embeds = []

for i in range(10) : 
    
    preprocessed_batch = preprocessed_tensor[i*250:(i+1)*250]
    preprocessed_batch = preprocessed_batch.to(device)

    # 4. Get the embeddings (feature maps)
    with torch.no_grad():  # Disable gradients for inference
        feature_maps = resnet(preprocessed_batch)  # Shape will be (batch_size, 512, 1, 1)

    # 5. Flatten the feature maps (optional)
    embeddings = feature_maps.view(feature_maps.size(0), -1)  # Flatten to shape (batch_size, embedding_size)

    # Print the embeddings shape (should be 2500 x 512)
    print("Embeddings shape:", embeddings.shape)
    
    embeds.append(embeddings)

Embeddings shape: torch.Size([250, 512])
Embeddings shape: torch.Size([250, 512])
Embeddings shape: torch.Size([250, 512])
Embeddings shape: torch.Size([250, 512])
Embeddings shape: torch.Size([250, 512])
Embeddings shape: torch.Size([250, 512])
Embeddings shape: torch.Size([250, 512])
Embeddings shape: torch.Size([250, 512])
Embeddings shape: torch.Size([250, 512])
Embeddings shape: torch.Size([250, 512])


In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics.pairwise import euclidean_distances
import torchvision.models as models

In [84]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torchvision import models, transforms
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

class LearningWithPrototype:
    def __init__(self, n_classes, feature_dim=512, learning_rate=0.001, model_name='resnet18', batch_size=32):
        self.n_classes = n_classes
        self.feature_dim = feature_dim
        self.learning_rate = learning_rate
        self.model_name = model_name
        self.batch_size = batch_size
        self.prototypes = None
        self.feature_extractor = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def initialize_model(self):
        """Initialize the pre-trained feature extractor"""
        resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove the last layer
        self.feature_extractor = resnet.to(self.device)  # Move model to GPU if available

    def extract_features(self, images):
        """Convert images to features using the feature extractor"""
        self.feature_extractor.eval()
        features_list = []
        
        X_tensor = torch.tensor(images, dtype=torch.float32).to(self.device)  # Move images to device

        transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize images to 224x224 (ResNet input size)
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
        ])

        tensor = X_tensor.float()
        processed_images = transform(tensor)
        
        # Create DataLoader for batch processing
        dataset = TensorDataset(processed_images)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)
        
        with torch.no_grad():
            for batch in dataloader:
                batch = batch[0].to(self.device)  # Move batch to device
                batch_features = self.feature_extractor(batch)
                features_list.append(batch_features)
        
        return torch.cat(features_list, dim=0).cpu().numpy().reshape(-1, self.feature_dim)  # Move final output to CPU for numpy compatibility

    def compute_prototypes(self, features, labels):
        """Compute class prototypes from features"""
        prototypes = np.zeros((self.n_classes, self.feature_dim))
        
        features = features.reshape(features.shape[0], self.feature_dim)
        
        for i in range(self.n_classes):
            if np.sum(labels == i) > 0:
                prototypes[i] = np.mean(features[labels == i], axis=0)
        return prototypes

    def predict(self, features):
        """Predict labels based on distance to prototypes"""
        distances = euclidean_distances(features, self.prototypes)
        return np.argmin(distances, axis=1)

    def fit_predict_iterate(self, datasets):
        """Iterative training and prediction on multiple datasets"""
        all_predictions = []
        
        # Initialize model
        self.initialize_model()
        
        # Initial feature extraction and prototype computation
        X_init, y_init = datasets[0]
        features_init = self.extract_features(X_init)
        self.prototypes = self.compute_prototypes(features_init, y_init)
        
        # Store predictions for first dataset
        all_predictions.append(y_init)
        
        # Iterate through remaining datasets
        for i, (X_next, _) in enumerate(datasets[1:], 1):
            print(f"\nProcessing dataset {i+1}")
            
            # Extract features for current dataset
            features_next = self.extract_features(X_next)
            
            # Predict labels using current prototypes
            predicted_labels = self.predict(features_next)
            all_predictions.append(predicted_labels)
            
            # Update prototypes with new features
            features_next = self.extract_features(X_next)
            self.prototypes = self.compute_prototypes(features_next, predicted_labels)
        
        return all_predictions


In [85]:
datasets = []

for i in range (1, 11) :
    train_path = os.path.join(train_dir, f'{i}_train_data.tar.pth')
    data = torch.load(train_path, weights_only = False)
    datasets.append((data['data'].reshape(-1,3,32,32), data['targets'] if 'targets' in data else np.zeros(data['data'].shape[0])))

In [86]:

"""
Process multiple datasets using Learning with Prototype with pre-trained feature extractor

Parameters:
datasets: List of tuples (X, y) where X is the image data and y is labels
            First dataset should have labels, others can have dummy labels
model_name: Name of the pre-trained model to use ('resnet18', 'resnet50', 
            'efficientnet_b0', or 'vit_b_16')
"""
# Initialize LwP model
n_classes = 10
lwp = LearningWithPrototype(
    n_classes=n_classes,
    batch_size=16
)

predictions = lwp.fit_predict_iterate(datasets)


Processing dataset 2

Processing dataset 3

Processing dataset 4

Processing dataset 5

Processing dataset 6

Processing dataset 7

Processing dataset 8

Processing dataset 9

Processing dataset 10


In [88]:
len(predictions)

10

In [91]:
for pred in predictions : 
    print(pred)

[6 9 9 ... 9 9 7]
[6 9 9 ... 8 6 5]
[4 6 7 ... 0 8 3]
[0 1 6 ... 1 2 5]
[6 2 5 ... 9 9 6]
[6 1 2 ... 3 6 7]
[2 5 3 ... 5 4 5]
[5 3 9 ... 7 0 8]
[8 1 5 ... 6 1 6]
[3 7 4 ... 2 9 5]
